In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap


import sys
sys.path.insert(0,'../')
import common.datautils as datautils

%matplotlib inline

In [ ]:
threads_data = datautils.load_csv_results('THREADS.results.csv', ['Benchmark','NumThreads', 'Success', 'RunId'])
threads_data.head()

In [ ]:
benchmarks = threads_data["Benchmark"].unique()
print(benchmarks)

In [ ]:
def compare():
    overview = []
    
    for benchmark in benchmarks:
        benchmark_data = threads_data[threads_data["Benchmark"] == benchmark]
        
        min_energy_threads = benchmark_data.loc[benchmark_data["Energy"].idxmin()]["NumThreads"]
        min_time_threads = benchmark_data.loc[benchmark_data["Time"].idxmin()]["NumThreads"]
        
        print(f"{benchmark} \t\t {min_energy_threads} \t {min_time_threads}")
        
compare()

In [ ]:
def one_thread_relative_data(data, benchmark):
    one_thread_data = data.loc[data["NumThreads"] == 1]
    
    relative_data = data.copy(deep=True)
    
    for benchmark in benchmarks:
        benchmark_one_thread_data = one_thread_data.loc[relative_data["Benchmark"] == benchmark]
        energy = benchmark_one_thread_data.iloc[0]["Energy"]
        time = benchmark_one_thread_data.iloc[0]["Time"]

        relative_data.loc[relative_data["Benchmark"] == benchmark, "Energy"] /= energy
        relative_data.loc[relative_data["Benchmark"] == benchmark, "Time"] /= time

    return relative_data

def double_plot_relative_to_one_thread(relative_data, benchmark):
    benchmark_data = relative_data[relative_data["Benchmark"] == benchmark]
    
    X = benchmark_data["NumThreads"].values
    
    energy = benchmark_data["Energy"].values
    time = benchmark_data["Time"].values
    

    plt.figure()
    plt.scatter(X, energy, marker='o', label='Energy')
    plt.scatter(X, time, marker='x', label='Time')
    
    plt.axhline(1, label='1 Core')
    
    plt.title('\n'.join(wrap(f'Improvement of Energy and Time relative to 1 core at different core counts for {benchmark}.', 55)))
    plt.ylabel('Relative to 1 Core')
    plt.xlabel('Number of cores')
    plt.xticks([x for x in np.arange(2, 29, 2)])
    plt.grid()
    plt.legend()

In [ ]:
relative_data = one_thread_relative_data(threads_data, benchmarks)

for benchmark in ['bfs', 'tpacf', 'botsalgn', 'botsspar']:
    double_plot_relative_to_one_thread(relative_data, benchmark)

In [ ]:
def plot_energy_vs_time(benchmark, data):
    benchmark_data = data.loc[data["Benchmark"] == benchmark]

    plt.figure(figsize=(12, 10))
    
    for num_threads in benchmark_data["NumThreads"].unique():
        thread_data = benchmark_data[benchmark_data["NumThreads"] == num_threads]
        
        X = thread_data["Time"].values
        Y = thread_data["Energy"].values

        plt.scatter(X, Y, label=f"{num_threads} threads")

    plt.title('Energy vs Time results for different thread counts on ' + benchmark)
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')

In [ ]:
# for benchmark in benchmarks:
#     plot_energy_vs_time(benchmark, threads_data)